In [13]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import steplib as msl
%matplotlib inline

In [14]:
PATH = "DataOx\\Hip_100Hz"

all_files = os.listdir(PATH)

print(all_files) 


['P01_hip100.csv', 'P02_hip100.csv', 'P03_hip100.csv', 'P04_hip100.csv', 'P05_hip100.csv', 'P06_hip100.csv', 'P07_hip100.csv', 'P08_hip100.csv', 'P09_hip100.csv', 'P10_hip100.csv', 'P11_hip100.csv', 'P12_hip100.csv', 'P13_hip100.csv', 'P14_hip100.csv', 'P15_hip100.csv', 'P16_hip100.csv', 'P17_hip100.csv', 'P18_hip100.csv', 'P19_hip100.csv', 'P20_hip100.csv', 'P21_hip100.csv', 'P22_hip100.csv', 'P23_hip100.csv', 'P24_hip100.csv', 'P25_hip100.csv', 'P26_hip100.csv', 'P27_hip100.csv', 'P28_hip100.csv', 'P29_hip100.csv', 'P30_hip100.csv', 'P31_hip100.csv', 'P32_hip100.csv', 'P33_hip100.csv', 'P34_hip100.csv', 'P35_hip100.csv', 'P36_hip100.csv', 'P37_hip100.csv', 'P38_hip100.csv', 'P39_hip100.csv']


In [15]:
import numpy as np
end = 60000

import time

def getSteps(file, window_size=20, threshold=1.15):
    start_time = time.time()
    file_data = pd.read_csv("DataOx\\Hip_100Hz\\" + file)[0:end]

    expected = np.where(file_data["annotation"] == 1)[0]

    data1 = file_data["x"] ** 2 + file_data["y"] ** 2 + file_data["z"] ** 2
    accelerometer = data1.apply(lambda x: x ** 0.5)
    data1 = msl.apply_filter(accelerometer, [0.0167855014066363,	0.0470217363830679,	0.121859539593060,	0.198782647391950,	0.231101150450572,	0.198782647391950,	0.121859539593060,	0.0470217363830679,	0.0167855014066363])
    data1 = msl.maximum_difference(data1, window_size)
    data1 = msl.detect_outliers(data1, threshold)
    data1 = msl.post_processing(data1, 250, 100)
    calculated = np.where(data1 == 1)[0]
    return calculated, expected , accelerometer


In [16]:
def chart(data_in):
    (calculated, expected, accelerometer) = data_in
    x=range(len(accelerometer))
    y=accelerometer
    plt.figure(figsize=(10,5))
    plt.plot(x,y, color='red', marker='o')
    plt.xlabel('index')
    plt.ylabel('magnitude')
    #show x label every 800
    
    #add rectangles
    for e in expected:
        plt.axvline(x =e, color ='g', label ='axvline - full height', ymin=0.5, alpha=0.8)
    for c in calculated:
        plt.axvline(x =c, color ='b', label ='axvline - full height', ymax=0.5, alpha=0.8)


    plt.show()

In [17]:
# differences = []
# i = 0
# for file in all_files:
#     (calculated1, expected1, _) = getSteps(file)
#     differences.append(len(expected1) - len(calculated1))
#     print(i, len(expected1), len(calculated1), file)
#     i += 1
# print(differences) 

In [18]:
steps = getSteps(all_files[28])

In [19]:
%matplotlib qt
chart(steps)

In [ ]:
from scipy.optimize import minimize

def oneObjective(x,i):
    result = getSteps(all_files[i], int(x[0]), x[1])
    return abs(len(result[1]) - len(result[0]))/(len(result[1])+1)

def objective(x):
    return sum([oneObjective(x, i) for i in range(0, 30)])



x0 = np.array([42, 1.75])
res = minimize(objective, x0, method='nelder-mead', options={'xatol': 1e-1, 'disp': True, 'maxiter': 30})
print(res.x)
print(res.fun)

In [40]:

for i in range(0, 10):
    print(oneObjective(res.x, i)*100)

27.958579881656803
26.47058823529412
0.7151370679380215
23.076923076923077
12.4031007751938
13.580246913580247
11.723329425556859
3.0303030303030303
7.920792079207921
1000.0


In [41]:
chart(getSteps(all_files[9],int(res.x[0]),res.x[1]))

In [ ]:
import time

def getSteps2(file, window_size=20, threshold=1.15):
    start_time = time.time()
    file_data = pd.read_csv("DataOx\\Hip_100Hz\\" + file)[0:end]
    print("Time taken for reading file: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    expected = np.where(file_data["annotation"] == 1)[0]
    print("Time taken for np.where: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = file_data["x"] ** 2 + file_data["y"] ** 2 + file_data["z"] ** 2
    accelerometer = data1.apply(lambda x: x ** 0.5)
    print("Time taken for calculating accelerometer: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.apply_filter(accelerometer, [0.0167855014066363,	0.0470217363830679,	0.121859539593060,	0.198782647391950,	0.231101150450572,	0.198782647391950,	0.121859539593060,	0.0470217363830679,	0.0167855014066363])
    print("Time taken for apply_filter: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.modified_pan_tompkins_scoring(data1, window_size)
    print("Time taken for modified_pan_tompkins_scoring: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.detect_outliers(data1, threshold)
    print("Time taken for detect_outliers: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.post_processing(data1, 250, 100)
    print("Time taken for post_processing: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    calculated = np.where(data1 == 1)[0]
    print("Time taken for final np.where: %s seconds" % (time.time() - start_time))

    return calculated, expected , accelerometer
